In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os

!ls -lh /content/drive/MyDrive/car_damage_detection

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import random

from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow import keras

# Load your trained model
MODEL_PATH = "/content/drive/MyDrive/car_damage_detection/MobileNet_Car_detection_FineTuned.keras"
model = keras.models.load_model(MODEL_PATH)
print("[INFO] Model loaded successfully!")

In [ ]:
# Define Class Names
CATEGORIES = ["00-damage", "01-whole"]

In [ ]:
#  Predict on a Training Image (sanity check)
# Randomly select one image from your training folders
train_dir = "/content/drive/MyDrive/car_damage_detection/data1a/training/"
categories = ["00-damage", "01-whole"]
category = random.choice(categories)
img_name = random.choice(os.listdir(os.path.join(train_dir, category)))
img_path = os.path.join(train_dir, category, img_name)

# Load and preprocess
image = load_img(img_path, target_size=(224, 224))
img_array = img_to_array(image)
img_array = np.expand_dims(img_array, axis=0)
img_array = (img_array / 127.5) - 1.0   # normalization for MobileNetV2
threshold = 0.38

# Predict
pred = model.predict(img_array)[0]
label = categories[0] if pred[0] > threshold else categories[1]

# Show
plt.imshow(image)
plt.axis("off")
plt.title(f"Actual: {category}, Predicted: {label}\nProbabilities: {np.round(pred, 3)}")
plt.show()

In [ ]:
#  Predict on a Validation Image (evaluation check)

val_dir = "/content/drive/MyDrive/car_damage_detection/data1a/validation"
categories = ["00-damage", "01-whole"]
category = random.choice(categories)
img_name = random.choice(os.listdir(os.path.join(val_dir, category)))
img_path = os.path.join(val_dir, category, img_name)

image = load_img(img_path, target_size=(224, 224))
img_array = img_to_array(image)
img_array = np.expand_dims(img_array, axis=0)
img_array = (img_array / 127.5) - 1.0

pred = model.predict(img_array)[0]
label = categories[0] if pred[0] > threshold else categories[1]

plt.imshow(image)
plt.axis("off")
plt.title(f"Actual: {category}, Predicted: {label}\nProbabilities: {np.round(pred, 3)}")
plt.show()

In [ ]:
#  Predict on a Web Image (real‑world test)

img_path = "/content/drive/MyDrive/car_damage_detection/web_test(1).jpg"

# Load and preprocess
image = load_img(img_path, target_size=(224, 224))
img_array = img_to_array(image)
img_array = np.expand_dims(img_array, axis=0)
img_array = (img_array / 127.5) - 1.0

# Predict
pred = model.predict(img_array)[0]
categories = ["00-damage", "01-whole"]
label = categories[0] if pred[0] > threshold else categories[1]

plt.imshow(image)
plt.axis("off")
plt.title(f"Predicted: {label}\nProbabilities: {np.round(pred, 3)}")
plt.show()

In [ ]:
# Batch Testing

TEST_FOLDER = "/content/drive/MyDrive/car_damage_detection/data1a/validation/00-damage"  # or 01-whole

for img_name in os.listdir(TEST_FOLDER):
    img_path = os.path.join(TEST_FOLDER, img_name)
    image = load_img(img_path, target_size=(224, 224))
    img_array = img_to_array(image)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = (img_array / 127.5) - 1.0

    pred = model.predict(img_array)
    label = CATEGORIES[0] if pred[0, 0] > threshold else CATEGORIES[1]

    plt.imshow(image)
    plt.title(f"Prediction: {label}")
    plt.axis("off")
    plt.show()

In [ ]:
# Compute Metrics (Across a Test Set)

from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

base_path = "/content/drive/MyDrive/car_damage_detection/data1a/validation"
data, labels = [], []

for category in CATEGORIES:
    folder = os.path.join(base_path, category)
    for fname in os.listdir(folder):
        fpath = os.path.join(folder, fname)
        img = load_img(fpath, target_size=(224, 224))
        x = img_to_array(img)
        x = preprocess_input(x)
        data.append(x)
        labels.append(category)

# Encode labels
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
labels_encoded = lb.fit_transform(labels)
labels_encoded = keras.utils.to_categorical(labels_encoded)

data = np.array(data, dtype="float32")
labels = np.array(labels_encoded)

# Predict
preds = model.predict(data)
y_true = np.argmax(labels, axis=1)
y_pred = np.where(preds[:, 0] > threshold, 0, 1)

print(classification_report(y_true, y_pred, target_names=CATEGORIES))

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import os

os.makedirs("results", exist_ok=True)

cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(5,4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=CATEGORIES,
            yticklabels=CATEGORIES)
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.tight_layout()
plt.savefig("results/confusion_matrix.png")
plt.show()
plt.close()
